<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%201/1_10_Ergebnisse_und_Auswertung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scikit-learn==1.0.0

In [ ]:
import os # u.a. zur Entwicklugn plattformübergreifender Systempfade
import pandas as pd # Datenmanagement
import numpy as np # Hilfsfunktionen für mathematische Operationen

# Datenvisualisierung
import seaborn as sns 
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split # Datensplits
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn import metrics # Modellevaluierung

## eigene Funktionen
def filter_df_by_proximity(df, proximity):
    return df.loc[df["ocean_proximity"] == proximity]

def engineer_features(df):
    df["ratio_bedrooms"] = df["total_bedrooms"] / df["total_rooms"]
    df["people_per_household"] = df["population"] / df["households"]
    return df

def get_features_and_targets(df):
    X = df.drop(["median_house_value"], axis=1).values
    y = np.stack(df["median_house_value"])
    return X, y

In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.getcwd()
FILE_PATH = "housing.csv"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

df = pd.read_csv(FILE_PATH) # Wir lesen die Datei housing.csv ein

df = df.dropna() # löscht alle Zeile mit fehlenden Attributen
df = df.reset_index(drop=True) # zählt unsere Daten neu durch

description = df.describe()

bins = [0] + list(description["median_house_value"][
    ["25%", "50%", "75%"]
].astype(int)) + [np.inf]

df["house_cat"] = pd.cut(
    df["median_house_value"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in split.split(df, df["house_cat"]):
    df_train = df.loc[train_index]
    df_test = df.loc[test_index]
    
df_train = df_train.drop("house_cat", axis=1)
df_test = df_test.drop("house_cat", axis=1)

df_train = df_train.drop(filter_df_by_proximity(df_train, "ISLAND").index)
df_test = df_test.drop(filter_df_by_proximity(df_test, "ISLAND").index)

df_train = engineer_features(df_train)
df_test = engineer_features(df_test)

df_train_ml = pd.get_dummies(df_train) # One-Hot Encoding
df_test_ml = pd.get_dummies(df_test)

X_train, y_train = get_features_and_targets(df_train_ml)
X_test, y_test = get_features_and_targets(df_test_ml)

clf = LinearRegression()
clf.fit(X_train, y_train)

# 1.10 Ergebnis und Auswertung

Wir können nun unser Modell für Prognosen verwenden:

In [ ]:
predictions = clf.predict(X_test)

Schauen wir uns die Prognosen unseres Modells einmal im Vergleich zu einigen echten Werten an:

In [ ]:
print("Prediction\t|True Value")
print("-"*30)
for idx, (pred, annotation) in enumerate(zip(predictions, y_test)):
    if idx == 10:
        break
    pred = int(pred)
    annotation = int(annotation)
    print(f"{pred}\t\t|{annotation}")

Auch hier sehen wir, dass das Modell z.T. starke Fehlprognosen liefert (mit fast 100.000€ Unterschied zum Realwert); an vielen Stellen liefert das Modell jedoch auch schon sehr gute Prognosen. Wir können einen High-Level Überblick erhalten, indem wir Kennzahlen berechnen:

$MAE(y, \hat{y}) = \frac{1}{n}\sum_{i=0}^{n}{|y_{i} - \hat{y}_{i}|}$

"Mit welcher Abweichung vom Realwert können wir durchschnittlich rechnen?"

$R^{2}(y, \hat{y}) = 1 - \frac{\frac{1}{n}\sum_{i=0}^{n}{|y_{i} - \hat{y}_{i}|}}{\frac{1}{n}\sum_{i=0}^{n}{|y_{i} - \overline{y}_{i}|}}$

"Wie viel der Varianz von y (unserem Zielwert) wird durch die unabhängigen Variablen des Modells erklärt"; stark vereinfacht: wenn der Wert = 100% ist, haben wir eine perfekte Prognose. Wenn er 0% ist, ist unsere Prognose genau so gut, wie der Mittelwert der Daten. Ein Wert < 0% (möglich) bedeutet, dass wir eine schlechtere Prognose als den Mittelwert geben.

In [ ]:
mae = metrics.mean_absolute_error(y_test, predictions)
mae_ratio = metrics.mean_absolute_percentage_error(y_test, predictions)
r2_score = metrics.r2_score(y_test, predictions)

In [ ]:
print(f"Wir können durchschnittlich mit einem Fehler von {np.round(mae)} rechnen;")
print(f"Das entspricht im Schnitt einer Fehlerquote von {np.round(mae_ratio * 100)}%;")
print(f"Der R^2 Werte liegt bei {np.round(r2_score * 100)}%")

Schauen wir uns das zuletzt noch an einem einzelnen Beispiel im Detail an:

In [ ]:
df_test.iloc[0]

In ML-Format sieht der Record wie folgt aus:

In [ ]:
X_test[0]

Und die Prognose ist:

In [ ]:
single_pred = clf.predict([X_test[0]])[0]
print(f"Die Prognose liegt bei {single_pred}.")
print(f"Der Realwert war {y_test[0]}.")